# Off-Line Automatic Differentiation

In this notebook, the *off-line automatic differentiation* idea is demonstrated.

In [1]:
using LinearAlgebra, Printf
import Base: +,*,^,sin,exp # we will extend these functions

struct Expression
	func::Function  
	deriv::Function 
end

Expression() = Expression(x->x, x->1.) # default constructor

Expression

In [2]:
# define +,*,^

+(e1::Expression,e2::Expression) = begin
	f1 = e1.func
	f2 = e2.func
	d1 = e1.deriv
	d2 = e2.deriv
	
	return Expression( x->f1(x)+f2(x), x->d1(x)+d2(x) )
end

*(e1::Expression,e2::Expression) = begin
	f1 = e1.func
	f2 = e2.func
	d1 = e1.deriv
	d2 = e2.deriv
    
	return Expression( x->f1(x)*f2(x), x->f1(x)*d2(x)+d1(x)*f2(x) )
end

^(e::Expression,a::T) where {T <: Real} = begin
	f = e.func
	d = e.deriv
    
	return Expression( x->f(x)^a, x->a*f(x)^(a-1)*d(x) )
end

^ (generic function with 67 methods)

In [3]:
# define sin, exp

sin(e::Expression) = begin
	f = e.func
	d = e.deriv
	
	return Expression( x->sin(f(x)), x->cos(f(x))*d(x) )
end

exp(e::Expression) = begin
	f = e.func
	d = e.deriv
	
	return Expression( x->exp(f(x)), x->exp(f(x))*d(x) )
end

exp (generic function with 15 methods)

In [4]:
f(x) = x*sin(x) + exp(x^2)   # user-defined function
df(x) = sin(x) + x*cos(x) + 2*x*exp(x^2)  # hand-coded derivative

ex = f(Expression())
f_auto = ex.func # recreated user-defined function
df_auto = ex.deriv # derivative of the user-defined function

# check if the constructed func and deriv are accurate
test_points = [-2.,-1.,0.,1.,2.]
f_err = norm( f.(test_points) .- f_auto.(test_points) , Inf )
df_err = norm( df.(test_points) .- df_auto.(test_points) , Inf )

println("Error in f = $f_err")
println("Error in df = $df_err")

Error in f = 0.0
Error in df = 0.0


In [8]:
# check if the constructed func and deriv are efficient

# we will use this a benchmark function
benchmark(f,arg...) = begin
    return -time() + begin
        for i=1:1000
            f(arg...)
        end
        time()
    end 
end

tf = benchmark(f,1.)
tf_auto = benchmark(f_auto,1.)

tdf = benchmark(df,1.)
tdf_auto = benchmark(df_auto,1.)

println("Table: Evaluation time*1000 - make sure to run twice")
println("________________________________")
println("   |  Hand-coded | Our Method  |")
@printf("f  | %7.5e | %7.5e |\n",tf,tf_auto)
@printf("df | %7.5e | %7.5e |\n",tdf,tdf_auto)
println("────────────────────────────────")

Table: Evaluation time*1000 - make sure to run twice
________________________________
   |  Hand-coded | Our Method  |
f  | 4.69685e-05 | 4.57764e-05 |
df | 5.81741e-05 | 5.79357e-05 |
────────────────────────────────
